<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Hyperband_with_Ray_Tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ray

In [ ]:
pip install -U tensorboardx

In [ ]:
from ray import tune
from ray.tune.schedulers import HyperBandScheduler
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define the training function
def train_mnist(config):
    model = nn.Sequential(
        nn.Linear(28*28, config["hidden_size"]),
        nn.ReLU(),
        nn.Linear(config["hidden_size"], 10)
    )
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])
    criterion = nn.CrossEntropyLoss()

    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('.', train=True, download=True, transform=transforms.ToTensor()),
        batch_size=32, shuffle=True)

    model.train()
    for epoch in range(2):
        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.view(-1, 28*28)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

        # Report the loss to Ray Tune
        tune.report(loss=loss.item())

# Define HyperBand scheduler and run Ray Tune
hyperband = HyperBandScheduler(
    max_t=10,
    reduction_factor=3,
    metric="loss",
    mode="min"
)
analysis = tune.run(
    train_mnist,
    config={
        "hidden_size": tune.choice([32, 64, 128]),
        "lr": tune.loguniform(1e-4, 1e-2)
    },
    scheduler=hyperband,
    num_samples=10
)

# Print the best hyperparameter configuration
print("Best config: ", analysis.get_best_config(metric="loss", mode="min"))